In [ ]:
!pip install pyradiomics

In [ ]:
import radiomics
from radiomics import featureextractor
import SimpleITK as sitk
import matplotlib.pyplot as plt
import os
import pandas as pd
import pywt
import cv2
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

my_drive = '/content/gdrive/My Drive/'
import sys
sys.path.append(my_drive)

Mounted at /content/gdrive


In [ ]:
data_dir = '/content/gdrive/My Drive/Brats 2018/MICCAI_BraTS_2018_Data_Training/'

In [ ]:
def fuseCoeff(cooef1, cooef2,cooef3, cooef4, method):

    if (method == 'mean'):
        cooef = (cooef1 + cooef2+ cooef3+ cooef4) / 4
    elif (method == 'min'):
        cooef = np.minimum(cooef1,cooef2)
        cooef = np.minimum(cooef,cooef3)
        cooef = np.minimum(cooef,cooef4)
    elif (method == 'max'):
        cooef = np.maximum(cooef1, cooef2)
        cooef = np.maximum(cooef, cooef3)
        cooef = np.maximum(cooef, cooef4)
    else:
        cooef = []

    return cooef


In [23]:
def preprocessor(grade):
  folder_fullpath = data_dir + grade + '/'
  folder_name =os.listdir(folder_fullpath)
  print(folder_name)
  
  grade_csv = grade + "_data.csv"

  if grade == "HGG":
    grade_type= 1
  elif grade == "LGG":
    grade_type= 0

  all_image_data = pd.DataFrame() #blank dataframe

  for folder_no in range(25):
  #for folder_no in range(5):
    rp= folder_fullpath+folder_name[folder_no] + '/'

    # print(rp)
    # print(folder_name)

    mri=os.listdir(rp)
    mri.sort()
    if len(mri) == 5:
      seg = sitk.ReadImage(rp+"/"+mri[1])#the segmented file
      seg = sitk.GetArrayFromImage(seg)
      width = 120
      height = 120
      segt1 = np.zeros((len(seg), width, height))

      for idx in range(len(seg)): 
        img = seg[idx, :, :]
        img_sm = cv2.resize(img, (width, height), interpolation=cv2.INTER_CUBIC)
        segt1[idx, :, :] = img_sm

      segt1=sitk.GetImageFromArray(segt1)#ROI 1 labelled as 1
      
      ma_arr = sitk.GetArrayFromImage(segt1)
      for l in (1,4):
        ma_arr[ma_arr == l] = 1

      segt2 = sitk.GetImageFromArray(ma_arr)
      segt2.CopyInformation(segt1)#ROI 1 and 4 labeled as 1


      ma_arr = sitk.GetArrayFromImage(segt1)
      for l in (1, 4, 2):
        ma_arr[ma_arr == l] = 1

      segt3 = sitk.GetImageFromArray(ma_arr)
      segt3.CopyInformation(segt1)#ROI 1,2,4 labeled as 1
      FUSION_METHOD = 'mean' # Can be 'min' || 'max || anything you choose according theory

      inp1= sitk.ReadImage(rp+"/"+mri[0])#path to Flair
      inp2= sitk.ReadImage(rp+"/"+mri[2])#Path to T1
      inp3 = sitk.ReadImage(rp+"/"+mri[3])#path to T1c
      inp4 = sitk.ReadImage(rp+"/"+mri[4])#path to T2

      # Read the two image
      I1 = sitk.GetArrayFromImage(inp1)
      I2 = sitk.GetArrayFromImage(inp2)
      I3 = sitk.GetArrayFromImage(inp3)
      I4 = sitk.GetArrayFromImage(inp4)

      # First: Do wavelet transform on each image
      wavelet = 'db1'
      cooef1 = pywt.wavedec2(I1[:,:], wavelet)
      cooef2 = pywt.wavedec2(I2[:,:], wavelet)
      cooef3 = pywt.wavedec2(I3[:,:], wavelet)
      cooef4 = pywt.wavedec2(I4[:,:], wavelet)

      # Second: for each level in both image do the fusion according to the desire option
      fusedCooef = []

      for i in range(len(cooef1)-1):

          # The first values in each decomposition is the apprximation values of the top level
          if(i == 0):

              fusedCooef.append(fuseCoeff(cooef1[0],cooef2[0], cooef3[0], cooef4[0], FUSION_METHOD))

          else:

              # For the rest of the levels we have tupels with 3 coeeficents
              c1 = fuseCoeff(cooef1[i][0], cooef2[i][0], cooef3[i][0], cooef4[i][0], FUSION_METHOD)
              c2 = fuseCoeff(cooef1[i][1], cooef2[i][1], cooef3[i][1], cooef4[i][1], FUSION_METHOD)
              c3 = fuseCoeff(cooef1[i][2], cooef2[i][2], cooef3[i][2], cooef4[i][2], FUSION_METHOD)

              fusedCooef.append((c1,c2,c3))

      # Third: After we fused the cooefficent we nned to transfor back to get the image
      fusedImage = pywt.waverec2(fusedCooef, wavelet)

      # Forth: normmalize values to be in uint8
      fusedImage = np.multiply(np.divide(fusedImage - np.min(fusedImage),(np.max(fusedImage) - np.min(fusedImage))),255)
      fusedImage = fusedImage.astype(np.uint8)

      #fusedImage=sitk.ReadImage(fusedImage)


      fusedImage=sitk.GetImageFromArray(fusedImage)

      extractor = radiomics.featureextractor.RadiomicsFeatureExtractor()
      result = extractor.execute(fusedImage, segt1, label=1)
      data= pd.DataFrame(columns=result.keys())
      data= data.iloc[:, 22:]
      for featureName in data.columns:
        data.loc[0, featureName]=result[featureName]

      result = extractor.execute(fusedImage, segt2, label=1)
      data1= pd.DataFrame(columns=result.keys())
      data1= data1.iloc[:, 22:]
      for featureName in data1.columns:
        data1.loc[0, featureName]=result[featureName]

      result = extractor.execute(fusedImage, segt3, label=1)
      data2= pd.DataFrame(columns=result.keys())
      data2= data2.iloc[:, 22:]
      for featureName in data2.columns:
        data2.loc[0, featureName]=result[featureName]

      
      
      #data1["GRADE"]= data2["GRADE"]= data["GRADE"]=grade_type #0-LGG, 1-HGG

      # data1["CASE"]= data2["CASE"]= data["CASE"]=folder_name[folder_no]    

      data1.columns = [str(col) + ' ROI 1' for col in data1.columns]
      data2.columns = [str(col) + ' ROI 2' for col in data2.columns]
      data.columns = [str(col) + ' ROI 3' for col in data.columns]

      data1.insert(0, 'GRADE', grade_type)    
      data2.insert(0, 'GRADE', grade_type)
      data.insert(0, 'GRADE', grade_type)


      datac=data1.merge(data2, how="inner", on=["GRADE"])
      datac=datac.merge(data, how="inner", on=["GRADE"])

      fused_data=datac

      
      fused_data.insert(0, 'CASE', folder_name[folder_no])

  #    fused_data.to_csv("/content/gdrive/My Drive/Brats 2018/MICCAI_BraTS_2018_Data_Training/" + grade_csv, mode='a')

      all_image_data=all_image_data.append(fused_data,ignore_index=True)
      print("\nProcessed Record No: {}\n".format(folder_no+1))

  return all_image_data

In [24]:
grade_name = "LGG"
processed_data = preprocessor(grade_name)

['case_1', 'case_2', 'case_3', 'case_4', 'case_5', 'case_6', 'case_7', 'case_8', 'case_9', 'case_10', 'case_11', 'case_12', 'case_13', 'case_14', 'case_15', 'case_16', 'case_17', 'case_18', 'case_19', 'case_20', 'case_21', 'case_22', 'case_23', 'case_24', 'case_25', 'case_26', 'case_27', 'case_28', 'case_29', 'case_30', 'case_31', 'case_32', 'case_33', 'case_34', 'case_35', 'case_36', 'case_37', 'case_38', 'case_39', 'case_40', 'case_41', 'case_42', 'case_43', 'case_44', 'case_45', 'case_46', 'case_47', 'case_48', 'case_49', 'case_50', 'case_51', 'case_52', 'case_53', 'case_54', 'case_55', 'case_56', 'case_57', 'case_58', 'case_59', 'case_60', 'case_61', 'case_62', 'case_63', 'case_64', 'case_65', 'case_66', 'case_67', 'case_68', 'case_69', 'case_70', 'case_71', 'case_72', 'case_73', 'case_74', 'case_75']


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated



Processed Record No: 1



GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated



Processed Record No: 2



...


Processed Record No: 23



GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated



Processed Record No: 24



GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated



Processed Record No: 25



In [25]:
processed_data
processed_data.to_csv(data_dir + grade_name+"_testing_data.csv")

In [ ]:
data1 = pd.read_csv(data_dir + "HGG_testing_data.csv")
data2 = pd.read_csv(data_dir + "LGG_testing_data.csv")

output=data1.append(data2, ignore_index=True)

output.drop(columns =['Unnamed: 0'], inplace= True)

output.to_csv('/content/gdrive/My Drive/Brats 2019/CLF_testing_data.csv')